In [1]:
import sys
sys.path.append('C:\\Program Files\\ASAP 2.2\\bin')

import multiresolutionimageinterface as mir
import cv2
from tqdm import tqdm_notebook
import os



In [2]:
# # The path can also be read from a config file, etc.
# OPENSLIDE_PATH = "C:\\Program Files\\openslide-win64-20231011\\bin"
# os.add_dll_directory(OPENSLIDE_PATH)
# import openslide

In [3]:
reader = mir.MultiResolutionImageReader()
annotation_list = mir.AnnotationList()
xml_repository = mir.XmlRepository(annotation_list)

In [9]:
dirAnnotations = '../data/annotations/'
dirData = '../data/training/'

In [10]:
ImageFiles = []
# r=root, d=directories, f = files
for r, d, f in os.walk(dirData):
    for file in f:
        if '.tif' in file and 'mask' not in file:
            ImageFiles.append(os.path.join(r, file))

In [11]:
def CreateAnnotationMask(annotationPath):
    
    # get only the name without dir or file suffix
    fileNamePart = annotationPath.replace('.xml','').replace(dirAnnotations, "")
    
    # what is the corresponding tif file - directories may vary so search from the list
    tifName = fileNamePart + '.tif'
    partialMatches = [s for s in ImageFiles if tifName in s]
    if len(partialMatches) == 0:
        print('Warning - This file is missing from the file list: {0} - skipping.'.format(tifName))
        return
    tifPath = partialMatches[0]
    
    # skip if tif file is not found
    if (not os.path.isfile(tifPath)): 
        print('Warning - Could not locate {0} - skipping this annotation file.'.format(tifPath))
        return
    
    # Skip if this mask is already found
    maskPath = tifPath.replace('.tif', '_mask.tif')
    if (os.path.isfile(maskPath)):
        print('Info - Mask file of {0} already exists - skipping'.format(tifPath))
        return
    
    # create mask
    xml_repository.setSource(annotationPath)
    xml_repository.load()
    annotation_mask = mir.AnnotationToMask()
    mr_image = reader.open(tifPath)
    if(mr_image is None):
        print('Warning - Could not read {0} - skipping'.format(tifPath))
        return
    label_map = {'metastases': 1, 'normal': 2}
    conversion_order = ['metastases', 'normal']
    annotation_mask.convert(annotation_list, 
                            maskPath, 
                            mr_image.getDimensions(), 
                            mr_image.getSpacing(), 
                            label_map, 
                            conversion_order)

In [12]:
AnnotationFiles = []
# r=root, d=directories, f = files
for r, d, f in os.walk(dirAnnotations):
    for file in f:
        if '.xml' in file:
            AnnotationFiles.append(os.path.join(r, file))

In [13]:
AnnotationFiles[:4]

['../data/annotations/patient_004_node_4.xml',
 '../data/annotations/patient_009_node_1.xml',
 '../data/annotations/patient_010_node_4.xml',
 '../data/annotations/patient_012_node_0.xml']

In [14]:
from tqdm.notebook import tqdm

for f in tqdm(AnnotationFiles, 'Creating masks...'):
    print('Annotation file: ' + f)
    CreateAnnotationMask(f)


Creating masks...:   0%|          | 0/50 [00:00<?, ?it/s]

Annotation file: ../data/annotations/patient_004_node_4.xml
Info - Mask file of ../data/training/center_0\patient_004_node_4.tif already exists - skipping
Annotation file: ../data/annotations/patient_009_node_1.xml
Warning - This file is missing from the file list: patient_009_node_1.tif - skipping.
Annotation file: ../data/annotations/patient_010_node_4.xml
Warning - This file is missing from the file list: patient_010_node_4.tif - skipping.
Annotation file: ../data/annotations/patient_012_node_0.xml
Warning - This file is missing from the file list: patient_012_node_0.tif - skipping.
Annotation file: ../data/annotations/patient_015_node_1.xml
Warning - This file is missing from the file list: patient_015_node_1.tif - skipping.
Annotation file: ../data/annotations/patient_015_node_2.xml
Warning - This file is missing from the file list: patient_015_node_2.tif - skipping.
Annotation file: ../data/annotations/patient_016_node_1.xml
Warning - This file is missing from the file list: pati